In [147]:
import fitz
from df2gspread import df2gspread as d2g
import pandas as pd
import gspread
from oauth2client.service_account import ServiceAccountCredentials
from pandas.io.json import json_normalize
import re
test=True

In [148]:
%ls

Annotate Code.ipynb             Jupyter_and_Google_Sheets.json
Breakdown Code.ipynb            README.md
Draft3-Full-annotated.pdf       all_annotation.html
Draft3-Full.pdf                 all_annotation.md
Draft3-Short.pdf                toc.csv
Example-Annotation.pdf


In [149]:
%cp Draft3-Full.pdf Draft3-Full-annotated.pdf

In [150]:
### READ IN PDF
annotated_doc = fitz.open("Draft3-Full-annotated.pdf")
original_doc = fitz.open("Draft3-Full.pdf")

page_width = annotated_doc[0].rect.x1
print page_width

612.0


In [151]:
print "here"

here


In [152]:
class Annotation:
    def __init__(self, page_number, text, comment, code, areas, footnote_index=None, url=None):
        self.page_number = int(page_number)
        self.text = text
        self.comment = comment
        self.code = code
        self.areas = areas
        self.footnote_index = footnote_index
        self.footnote_location = (0,0)
        self.page_level = True
        self.url = url
    
    def __repr__(self):
        return "{} - {}".format(self.page_number, self.footnote_index)
    
    def find_note_location(self, doc):
        page = doc[self.page_number-1]
        text_instances = page.searchFor(self.text)

        ## Find the inst
        inst = False
        for inst in text_instances:
            inst
        if inst:
            self.footnote_location = (inst.x1, inst.y0)
            self.page_level = False
        return self.footnote_location
    
    def set_page_level_location(self):
        if self.page_level:
            x = 40 * self.footnote_index
            y = 5
            self.footnote_location = (x, y)
        return self.footnote_location
    
    def add_to_pdf(self, doc):
        """
        Adds the given annotation to the given PDF doc
        """
        page = doc[self.page_number-1]
        text_instances = page.searchFor(self.text)

        ## Add the highlight
        inst = False
        for inst in text_instances:
            highlight = page.addHighlightAnnot(inst)

        footnote_loc = fitz.Rect(self.footnote_location[0],self.footnote_location[1]-2,self.footnote_location[0]+10,self.footnote_location[1]+6)
        footnote = page.addFreetextAnnot(footnote_loc, str(self.footnote_index), fontsize=8)
        link = {'kind': 2, 
                'from': fitz.Rect(footnote_loc.x0-1, footnote_loc.y0-1, footnote_loc.x1+1, footnote_loc.y1+1),
                'type': 'uri',
                'uri': self.url}
        page.insertLink(link)
        if self.page_level:
            comment = page.addTextAnnot(
                (footnote_loc.x0+5,footnote_loc.y0), 
                "{}. {}".format(self.footnote_index, self.comment), 
                icon="Comment")
        else:
            if footnote_loc.x0 < page_width/2:
                comment = page.addTextAnnot(
                    (10,footnote_loc.y0), 
                    "{}. {}".format(self.footnote_index, self.comment), 
                    icon="Comment")
            else:
                comment = page.addTextAnnot(
                    (page_width-40,footnote_loc.y0), 
                    "{}. {}".format(self.footnote_index, self.comment), 
                    icon="Comment")

annotation = Annotation(page_number=233, text="(9.4×Number of Units×Residents per Unit)/1,000=Acres of Parkland", 
                          comment="Would this apply to site-plan lites?", 
                          code="Error", 
                          areas=["Missing Middle", "Parkland"],
                            url="https://htmlpreview.github.io/?https://github.com/kmclaugh/land-development-code-review/blob/master/all_annotation.html")
annotation.find_note_location(annotated_doc)


(453.4897155761719, 676.2694702148438)

In [153]:
## Connect to Google Sheet
## Connect to our service account
scope = ['https://spreadsheets.google.com/feeds']
credentials = ServiceAccountCredentials.from_json_keyfile_name('./Jupyter_and_Google_Sheets.json', scope)
gc = gspread.authorize(credentials)
spreadsheet_key = '1R8L9HOvICuRiMJ3hAGGa1L709z4u_487D_QoOpVm57U'
book = gc.open_by_key(spreadsheet_key)
worksheet = book.worksheet("Form Responses 1")
table = worksheet.get_all_values()

##Convert table data into a dataframe
cdf = pd.DataFrame(table[1:], columns=table[0])

##Convert number strings to floats and ints
#cdf = cdf.apply(pd.to_numeric, errors='ignore')

##Convert date strings to datetime format
cdf['Timestamp'] = pd.to_datetime(cdf['Timestamp'],infer_datetime_format=True)

##Drop no timestamps
cdf = cdf[pd.notnull(cdf['Timestamp'])]


In [154]:
cdf

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Effected Areas,Reviewed?,Added
0,2019-09-07 14:29:45,kevin.mclaughlin70@gmail.com,233,"(9.4×Number of Units×Residents per Unit)/1,000...",It's possible that the total required parkland...,Error,Parkland,Yes,
1,2019-09-07 14:42:55,kevin.mclaughlin70@gmail.com,233,(C)\nSite Plan Dedication.,Would this apply to site-plan lites?,Question,"Missing Middle, Parkland",Yes,
2,2019-09-12 20:00:04,lwimberley@gmail.com,487,This zone allows detached housing. It can be s...,bleh,Negative Feedback,everything,Yes,
3,2019-09-12 20:26:23,lwimberley@gmail.com,482,"Other Allowed Uses\n—\n45'\n5,000 sf",meh,Negative Feedback,everything,Yes,
4,2019-09-12 20:29:42,lwimberley@gmail.com,485,Impervious Cover\n45% 3\n23-3D-3,sucks,yuk,Parkland,Yes,
5,2019-09-12 20:31:47,lwimberley@gmail.com,485,Personal Open Space\nNone required,yeah sure,dumb,bleh,Yes,
6,2019-09-13 06:35:00,kevin@kevincmclaughlin.com,233,,"Lorem ipsum dolor sit amet, consectetur adipis...",Question,Parkland,,
7,2019-09-13 06:36:05,kevin@kevincmclaughlin.com,233,,"Ut enim ad minim veniam, quis nostrud exercita...",Positive Feedback,Water Qaulity,,


In [155]:
cdf["Annotation"] = cdf.apply(lambda x: Annotation(page_number=x["Page Number"], text=x["Text"], comment=x["Comment"], code=x["Comment Codes"], areas=x["Effected Areas"]), axis=1)
cdf["Footnote Location"] = cdf.apply(lambda x: x["Annotation"].find_note_location(annotated_doc), axis=1)
cdf["Footnote Locationx"] = cdf.apply(lambda x: x["Footnote Location"][0], axis=1)
cdf["Footnote Locationy"] = cdf.apply(lambda x: x["Footnote Location"][1], axis=1)

cdf = cdf.sort_values(["Page Number", "Footnote Locationy", "Footnote Locationx"], ascending=[True, True, False])


In [156]:
## Find footnote indexes
current_index = 0
current_page = cdf.iloc[0]["Page Number"]
for index, row in cdf.iterrows():
    this_page = row["Page Number"]
    if current_page == this_page:
        current_index += 1
    else:
        current_page = this_page
        current_index = 1
    row["Annotation"].footnote_index = current_index
cdf["Footnote Index"] = cdf["Annotation"].map(lambda x: x.footnote_index)
cdf.head()

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Effected Areas,Reviewed?,Added,Annotation,Footnote Location,Footnote Locationx,Footnote Locationy,Footnote Index
6,2019-09-13 06:35:00,kevin@kevincmclaughlin.com,233,,"Lorem ipsum dolor sit amet, consectetur adipis...",Question,Parkland,,,233 - 1,"(0, 0)",0.000000,0.000000,1
7,2019-09-13 06:36:05,kevin@kevincmclaughlin.com,233,,"Ut enim ad minim veniam, quis nostrud exercita...",Positive Feedback,Water Qaulity,,,233 - 2,"(0, 0)",0.000000,0.000000,2
1,2019-09-07 14:42:55,kevin.mclaughlin70@gmail.com,233,(C)\nSite Plan Dedication.,Would this apply to site-plan lites?,Question,"Missing Middle, Parkland",Yes,,233 - 3,"(252.202026367, 388.762451172)",252.202026,388.762451,3
0,2019-09-07 14:29:45,kevin.mclaughlin70@gmail.com,233,"(9.4×Number of Units×Residents per Unit)/1,000...",It's possible that the total required parkland...,Error,Parkland,Yes,,233 - 4,"(453.489715576, 676.269470215)",453.489716,676.269470,4
3,2019-09-12 20:26:23,lwimberley@gmail.com,482,"Other Allowed Uses\n—\n45'\n5,000 sf",meh,Negative Feedback,everything,Yes,,482 - 1,"(328.787658691, 399.644165039)",328.787659,399.644165,1


In [157]:
## Update location for page level comments
cdf["Footnote Location"] = cdf.apply(lambda x: x["Annotation"].set_page_level_location(), axis=1)
cdf["Footnote Locationx"] = cdf.apply(lambda x: x["Footnote Location"][0], axis=1)
cdf["Footnote Locationy"] = cdf.apply(lambda x: x["Footnote Location"][1], axis=1)

In [158]:
cdf.apply(lambda x: x["Annotation"].add_to_pdf(annotated_doc), axis=1)
annotated_doc.save(annotated_doc.name, incremental=True, encryption=0)

In [159]:
cdf

,Timestamp,Email Address,Page Number,Text,Comment,Comment Codes,Effected Areas,Reviewed?,Added,Annotation,Footnote Location,Footnote Locationx,Footnote Locationy,Footnote Index
6,2019-09-13 06:35:00,kevin@kevincmclaughlin.com,233,,"Lorem ipsum dolor sit amet, consectetur adipis...",Question,Parkland,,,233 - 1,"(40, 5)",40.000000,5.000000,1
7,2019-09-13 06:36:05,kevin@kevincmclaughlin.com,233,,"Ut enim ad minim veniam, quis nostrud exercita...",Positive Feedback,Water Qaulity,,,233 - 2,"(80, 5)",80.000000,5.000000,2
1,2019-09-07 14:42:55,kevin.mclaughlin70@gmail.com,233,(C)\nSite Plan Dedication.,Would this apply to site-plan lites?,Question,"Missing Middle, Parkland",Yes,,233 - 3,"(252.202026367, 388.762451172)",252.202026,388.762451,3
0,2019-09-07 14:29:45,kevin.mclaughlin70@gmail.com,233,"(9.4×Number of Units×Residents per Unit)/1,000...",It's possible that the total required parkland...,Error,Parkland,Yes,,233 - 4,"(453.489715576, 676.269470215)",453.489716,676.269470,4
3,2019-09-12 20:26:23,lwimberley@gmail.com,482,"Other Allowed Uses\n—\n45'\n5,000 sf",meh,Negative Feedback,everything,Yes,,482 - 1,"(328.787658691, 399.644165039)",328.787659,399.644165,1
5,2019-09-12 20:31:47,lwimberley@gmail.com,485,Personal Open Space\nNone required,yeah sure,dumb,bleh,Yes,,485 - 1,"(487.931304932, 338.804168701)",487.931305,338.804169,1
4,2019-09-12 20:29:42,lwimberley@gmail.com,485,Impervious Cover\n45% 3\n23-3D-3,sucks,yuk,Parkland,Yes,,485 - 2,"(298.315002441, 338.97668457)",298.315002,338.976685,2
2,2019-09-12 20:00:04,lwimberley@gmail.com,487,This zone allows detached housing. It can be s...,bleh,Negative Feedback,everything,Yes,,487 - 1,"(476.35925293, 132.012451172)",476.359253,132.012451,1


In [160]:
annotated_doc[0].rect.x1/2

306.0